In [20]:
import numpy as np
import scipy.special
import scipy.optimize

class LinearLogisticRegression:

    def __init__(self):
        pass

    def __compute_zi(self, ci):
        return 2 * ci - 1

    def __logreg_obj(self, v):  # still works if DTR is one sample only? yes but it must be of shape (4,1)
        w, b = v[0:-1], v[-1]
        J = self.lbd / 2 * (np.linalg.norm(w) ** 2)
        summary = 0
        for i in range(self.N):
            xi = self.Dtrain[:, i:i + 1]
            ci = self.Ltrain[i]
            zi = self.__compute_zi(ci)
            summary += np.logaddexp(0, -zi * (np.dot(w.T, xi) + b))
        J += (1 / self.N) * summary
        return J

    def train(self, Dtrain, Ltrain, lbd, maxiter):
        self.Dtrain = Dtrain
        self.Ltrain = Ltrain
        self.lbd = lbd
        self.maxiter = maxiter
        self.F = Dtrain.shape[0]  # dimensionality of features space
        self.K = len(set(Ltrain))  # number of classes
        self.N = Dtrain.shape[1]
        self.x, f, d = scipy.optimize.fmin_l_bfgs_b(func=self.__logreg_obj,
                                                    x0=np.zeros(self.Dtrain.shape[0] + 1),
                                                    approx_grad=True,
                                                    iprint=0,
                                                    maxiter=self.maxiter)
        print('Point of minimum: %s' % (self.x))
        print('Value of the minimum: %s' % (f))
        print('Number of iterations: %s' % (d['funcalls']))
        return self

    def predict(self, Dtest, labels=True):
        w, b = self.x[0:-1], self.x[-1]
        S = np.zeros((Dtest.shape[1]))
        for i in range(Dtest.shape[1]):
            xi = Dtest[:, i:i + 1]
            s = np.dot(w.T, xi) + b
            S[i] = s
        if labels:
            LP = S > 0
            return LP
        else:
            return S
class QuadraticLogisticRegression:
    def __init__(self):
        pass

    def __compute_zi(self, ci):
        return 2 * ci - 1

    def __logreg_obj(self, v):  # still works if DTR is one sample only? yes but it must be of shape (4,1)
        w, b = v[0:-1], v[-1]
        J = self.lbd / 2 * (np.linalg.norm(w) ** 2)
        summary = 0
        for i in range(self.N):
            xi = self.Dtrain[:, i:i + 1]
            ci = self.Ltrain[i]
            zi = self.__compute_zi(ci)
            summary += np.logaddexp(0, -zi * (np.dot(w.T, xi) + b))
        J += (1 / self.N) * summary
        return J

    def train(self, Dtrain, Ltrain, lbd, maxiter):
        self.Dtrain = Dtrain
        self.Ltrain = Ltrain
        self.lbd = lbd
        self.maxiter = maxiter
        self.F = Dtrain.shape[0]  # dimensionality of features space
        self.K = len(set(Ltrain))  # number of classes
        self.N = Dtrain.shape[1]
        DTR_ext = numpy.hstack([self.__expandFeatures(self.Dtrain[:, i]) for i in range(self.Dtrain.shape[1])])
        self.x, f, d = scipy.optimize.fmin_l_bfgs_b(func=self.__logreg_obj, 
                                                    x0=numpy.zeros(DTR_ext.shape[0] + 1), 
                                                    approx_grad = True, 
                                                    factr=1.0)
        print('Point of minimum: %s' % (self.x))
        print('Value of the minimum: %s' % (f))
        print('Number of iterations: %s' % (d['funcalls']))
        return self    
    
    def __expandFeatures(x):
        x = utils.mcol(x)
        expX = utils.mcol(numpy.dot(x, x.T))
        return numpy.vstack([expX, x])

    def predict(self, Dtest):
        DTE_ext = numpy.hstack([self.__expandFeatures(Dtest[:, i]) for i in range(Dtest.shape[1])])
        w, b = utils.mcol(self.x[0:-1]), self.x[-1]
        scores = numpy.dot(w.T, DTE_ext) + b
        return scores[0, :]

In [21]:
def load_iris_binary():
    D, L = sklearn.datasets.load_iris()['data'].T, sklearn.datasets.load_iris()['target']
    D = D[:, L != 0] # remove setosa from D
    L = L[L!=0] # remove setosa from L
    L[L==2] = 0 # We assign label 0 to virginica (was label 2)
    return D, L
def split_db_2to1(D, L, seed=0):
    nTrain = int(D.shape[1]*2.0/3.0) # 2/3 of the dataset D are used for training, 1/3 for validation
    np.random.seed(seed)
    idx = np.random.permutation(D.shape[1]) # take a random array of 150 elements, each element is 0<x<=149 (np.arange(150))
    idxTrain = idx[0:nTrain] # first 100 are indices of training samples
    idxTest = idx[nTrain:] # remaining 50 are indices of validation samples
    DTR = D[:, idxTrain] # D for training
    DTE = D[:, idxTest] # D for validation
    LTR = L[idxTrain] # L for training
    LTE = L[idxTest] # L for validation
    return (DTR, LTR), (DTE, LTE)
DT, LT = load_iris_binary()
(DTR, LTR), (DTE, LTE) = split_db_2to1(D, L)

In [22]:
lbd = 10**-3
maxiter=100
model = LinearLogisticRegression()
labs = model.train(DTR, LTR, lbd, maxiter).predict(DTE, labels=True)
print('Error rate: ', 1 - (sum(labs == LTE) / len(LTE)))

Point of minimum: [ 1.72972493  0.98292107 -4.54959632 -7.12481584 20.95265108]
Value of the minimum: [0.1100009]
Number of iterations: 258
Error rate:  0.08823529411764708


In [ ]:
lbd = 10**-3
maxiter=100
model = QuadraticLogisticRegression()
labs = model.train(DTR, LTR, lbd, maxiter).predict(DTE, labels=True)
print('Error rate: ', 1 - (sum(labs == LTE) / len(LTE)))